In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
from matplotlib import style
import sklearn
from sklearn import linear_model
from sklearn.cluster import KMeans
from sklearn import preprocessing, cross_validation
from sqlalchemy import create_engine
import psycopg2


/Users/zhaoningbin/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# import data from PGSQL
engine=create_engine('postgresql://)

table_names=engine.table_names()
print (table_names)



['team_member', 'candidate_summary_join', 'census_bus_employee', 'census_social_0617', 'committee_linkage_join', 'master_join', 'master_join1', 'fec_summary', 'fec_operating_expenditure', 'cq_race', 'cq_candidates_race', 'fec_committee', 'fec_candidate', 'test_join', 'investigate', 'cross_walk', 'fec_candidate_committee_linkage', 'fec_individual_contribution', 'cq_race_incumbent', 'fec_committee_to_candidate_contribution', 'fec_committee_to_committee_contribution', 'fec_committee_2012', 'fec_table_join', 'test_join_win', 'candidate_join', 'fec_join', 'individual_contribution_join']


In [3]:
con=engine.connect()
rs1=con.execute('SELECT * FROM cq_candidates_race')

cq_candidates_race = pd.DataFrame(rs1.fetchall())
df = pd.DataFrame(cq_candidates_race)
cq_candidates_race.columns = rs1.keys()

con.close()

In [4]:
cq_candidates_race.head()

,race_uid,state,year,districtnumber,redistricteddate,totalvote,winingplurality,winningparty,incumbentwin,race_uid2,candidatename,incumbentstatus,candidateparty,votenumberreceived,voteproportionreceived
0,2014AR1,AR,2014,1,1/1/12,0,60584,R,Yes,2014AR1,"Crawford, Rick",Y,R,124139.0,63.25
1,2012AR1,AR,2012,1,1/1/12,0,42199,R,Yes,2012AR1,"Crawford, Rick",Y,R,138800.0,56.23
2,2014AR3,AR,2014,3,1/1/12,0,112325,R,Yes,2014AR3,"Womack, Steve",Y,R,151630.0,79.41
3,2014AR2,AR,2014,2,1/1/12,0,19596,R,No,2014AR2,"Hill, French",N,R,123073.0,51.86
4,2014AR4,AR,2014,4,1/1/12,0,23047,R,No,2014AR4,"Westerman, Bruce",N,R,110789.0,53.75


In [5]:
cq_candidates_race.drop(['candidatename','state','districtnumber','redistricteddate','race_uid2'], axis=1, inplace=True) 


In [6]:
cq_candidates_race.head()

,race_uid,year,totalvote,winingplurality,winningparty,incumbentwin,incumbentstatus,candidateparty,votenumberreceived,voteproportionreceived
0,2014AR1,2014,0,60584,R,Yes,Y,R,124139.0,63.25
1,2012AR1,2012,0,42199,R,Yes,Y,R,138800.0,56.23
2,2014AR3,2014,0,112325,R,Yes,Y,R,151630.0,79.41
3,2014AR2,2014,0,19596,R,No,N,R,123073.0,51.86
4,2014AR4,2014,0,23047,R,No,N,R,110789.0,53.75


In [7]:
def handle_non_numerical_data(df):
    columns = df.columns.values
    
    for column in columns:
        text_digit_vals = {}
        def convert_to_int(val):
            return text_digit_vals[val]
        
        if df[column].dtype !=np.int64 and df[column].dtype !=np.float64:
            column_contents = df[column].values.tolist()
            unique_elements = set(column_contents)
            x=0
            for uniqure in unique_elements:
                if unique not in text_digit_vals:
                    text_digit_vals[unique] = x
                    x+=1
                    
            df[column] = list(map(convert_to_int,df[column]))
            
    return df

df = handle_non_numerical_data(df)
print(df.head())

NameError: name 'unique' is not defined

In [8]:
df_x=pd.DataFrame(cq_candidates_race,columns=cq_candidates_race.candidateparty)
df_y=pd.DataFrame(cq_candidates_race.incumbentwin)

In [9]:
df_x.describe()

candidateparty,R,R,R,R,R,R,R,R,R,R,...,Independent,Independent,Independent,Libertarian,Libertarian,Libertarian,Libertarian,Libertarian,None,None
count,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
